In [12]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [13]:
data = pd.read_csv('data/learningSet.csv')
data.columns = data.columns.str.lower()
data.columns = data.columns.str.replace(" ","_")
data.shape

/Users/ttrofff/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(95412, 481)

In [14]:
data.dtypes.value_counts()

int64      310
float64     97
object      74
dtype: int64

In [15]:
categorical = data.select_dtypes('object')
categorical.shape

(95412, 74)

In [16]:
# Check for null values in all the columns

nulls = pd.DataFrame(categorical.isna().sum()/len(categorical)).reset_index()
nulls.columns = ['column','share']
nulls[nulls['share']>0]

,column,share
73,geocode2,0.001383


In [17]:
# Exclude the following variables by looking at the definitions. 
# Create a new empty list called drop_list. We will append this list and then drop all the columns 
# in this list later
## OSOURCE - symbol definitions not provided, too many categories
## ZIP CODE - we are including state already

drop_list = []
drop_list.append('osource')
drop_list.append('zip')
drop_list

['osource', 'zip']

In [18]:
# Identify columns that over 85% missing values

def trashhold_na(df,trashhold=0.25):
    
    nulls = pd.DataFrame(df.isna().sum()/len(data)).reset_index()
    nulls.columns = ['column','share']
    nulls_above_tr_db = nulls[nulls['share']>trashhold]
    nulls_above_tr_list = list(nulls_above_tr_db['column'])
    
    ## should we include columns with white spaces?
    #blanks = pd.DataFrame(data.isin([' ']).sum()/len(data)).reset_index()
    #blanks.columns = ['column','share']
    #blanks_above_tr_db = blanks[blanks['share']>trashhold]
    #blanks_above_tr_list = list(blanks_above_tr_db['column'])
    
    above_tr_list = nulls_above_tr_list# + blanks_above_tr_list
    
    return above_tr_list

columns_low_data = trashhold_na(categorical,0.85)
columns_low_data

[]

In [8]:
# lets check for blanks

def trashhold_bl(df,trashhold=0.25):
    
    blanks = pd.DataFrame(df.isin([' ']).sum()/len(data)).reset_index()
    blanks.columns = ['column','share']
    blanks_above_tr_db = blanks[blanks['share']>trashhold]
    blanks_above_tr_list = list(blanks_above_tr_db['column'])
    return blanks_above_tr_list

columns_low_data_blanks = trashhold_bl(categorical,0.85)
columns_low_data_blanks

['mailcode',
 'pvastate',
 'recinhse',
 'recp3',
 'recpgvg',
 'recsweep',
 'child03',
 'child07',
 'child12',
 'child18',
 'solp3',
 'solih',
 'major',
 'collect1',
 'veterans',
 'bible',
 'catlg',
 'homee',
 'cdplay',
 'stereo',
 'pcowners',
 'photo',
 'crafts',
 'fisher',
 'gardenin',
 'boats',
 'walker',
 'kidstuff',
 'cards',
 'plates']

In [19]:
# Remove those columns from the dataframe
# only NAs
categorical = categorical.drop(columns_low_data, axis = 1)
categorical.shape

(95412, 74)

In [20]:
# Reduce the number of categories in the column GENDER. 
# The column should only have either "M" for males, "F" for females, and "other" for all the rest

# Note that there are a few null values in the column. 
# We will first replace those null values using the code below:

print(categorical['gender'].value_counts())
# categorical['gender'].isna().sum() - zero NA
# categorical['GENDER'] = categorical['GENDER'].fillna('F')
# categorical['gender'].isin([' ']).sum() - but some are blank spaces, should we replace them with F?

categorical['gender'] = categorical['gender'].str.replace(' ','F')
categorical['gender'] = np.where(~categorical['gender'].isin(['M','F']) , 'other', categorical['gender'])
print(categorical['gender'].value_counts())


F    51277
M    39094
      2957
U     1715
J      365
C        2
A        2
Name: gender, dtype: int64
F        54234
M        39094
other     2084
Name: gender, dtype: int64
